## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-17-02-16-10 +0000,nypost,NJ ‘Moms for Liberty’ leader claims Girl Scout...,https://nypost.com/2025/08/16/us-news/nj-moms-...
1,2025-08-17-01-57-09 +0000,nyt,Trump Backs Plan to Cede Land for Peace in Ukr...,https://www.nytimes.com/live/2025/08/16/world/...
2,2025-08-17-01-45-32 +0000,missionlocal,Day 158 at the 16th Street Plaza: A tale of tw...,https://missionlocal.org/2025/08/16th-mission-...
3,2025-08-17-01-37-18 +0000,nypost,Couple gets married in midst of wild anti-Isra...,https://nypost.com/2025/08/16/us-news/couple-g...
4,2025-08-17-01-12-14 +0000,cbc,3 states sending hundreds more troops to U.S. ...,https://www.cbc.ca/news/world/national-guard-w...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,48
53,putin,31
18,ukraine,21
267,strike,12
266,canada,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
5,2025-08-17-01-10-58 +0000,nypost,First Lady Melania Trump calls for end to war ...,https://nypost.com/2025/08/16/world-news/first...,137
70,2025-08-16-18-48-22 +0000,nypost,How Trump’s meeting with Putin pivoted from Uk...,https://nypost.com/2025/08/16/us-news/inside-h...,133
152,2025-08-16-08-01-18 +0000,nyt,Trump and Putin had an isolated conversation i...,https://www.nytimes.com/live/2025/08/15/world/...,132
37,2025-08-16-21-22-09 +0000,nyt,Trump Backs Off Cease-Fire Demand in Ukraine W...,https://www.nytimes.com/2025/08/16/world/europ...,131
166,2025-08-16-03-36-11 +0000,nyt,"At Trump’s Summit, No Deal on Ukraine, and No ...",https://www.nytimes.com/2025/08/15/us/politics...,128


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
5,137,2025-08-17-01-10-58 +0000,nypost,First Lady Melania Trump calls for end to war ...,https://nypost.com/2025/08/16/world-news/first...
49,60,2025-08-16-20-15-27 +0000,wapo,Canada orders Air Canada flight attendants bac...,https://www.washingtonpost.com/world/2025/08/1...
58,50,2025-08-16-19-41-53 +0000,nypost,Trump’s efforts at Alaska summit applauded by ...,https://nypost.com/2025/08/16/world-news/trump...
87,31,2025-08-16-16-37-41 +0000,nypost,Zelensky warns Putin will step up attacks on U...,https://nypost.com/2025/08/16/world-news/zelen...
42,30,2025-08-16-20-57-24 +0000,wapo,Republican-led states to send hundreds more Na...,https://www.washingtonpost.com/nation/2025/08/...
82,30,2025-08-16-17-06-24 +0000,nypost,NYPD drone allows Mayor Adams to see real-time...,https://nypost.com/2025/08/16/us-news/drone-al...
28,25,2025-08-16-22-11-19 +0000,nypost,BB gun-wielding man fatally shot by off-duty c...,https://nypost.com/2025/08/16/us-news/bb-gun-w...
70,25,2025-08-16-18-48-22 +0000,nypost,How Trump’s meeting with Putin pivoted from Uk...,https://nypost.com/2025/08/16/us-news/inside-h...
27,23,2025-08-16-22-56-53 +0000,nypost,Louisiana bar busted for allegedly selling ove...,https://nypost.com/2025/08/16/us-news/louisian...
165,22,2025-08-16-04-01-06 +0000,nyt,"Once Seen as Divisive, South Korea’s New Leade...",https://www.nytimes.com/2025/08/16/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
